In [1]:
%matplotlib inline
import tensorflow.keras.backend as K
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score,confusion_matrix,accuracy_score
from sklearn.decomposition import  PCA,TruncatedSVD,FastICA
from tensorflow.keras.applications import VGG16, ResNet50,VGG19,ResNet50V2,EfficientNetB0
from sklearn.preprocessing import StandardScaler
from Preprocess import *
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import soundfile as sf
from pathlib import *
import pandas as pd
import numpy as np
import librosa
import glob
import shutil
import os
import cv2


In [2]:
INPUT_PATH = Path('/net/store/cbc/projects/Pan troglodytes/audio_PH_dataset/PH/good_data/')
file_path =  '/net/store/cv/users/ybrima/scratch/data/Luango_Speaker/clip_loango.npz'
file_path2 =  '/net/store/cv/users/ybrima/scratch/data/Luango_Speaker/clip_loango_resized.npz'
FRAME_LENGHT =  1024
SAMPLE_RATE =  44100
HOP_LENGTH =  512

In [3]:
CLASSES =  get_classes(INPUT_PATH)

In [4]:
# data =  np.load(file_path, allow_pickle=True)
# X = data['x']
# y = data['y']
# Z =  data['z']
# CLASSES = list(data['c'])

In [5]:
X,y,Z = build_dataset(INPUT_PATH,keepdims=False,crop_dims= (128,128),outpath=file_path2 )

1761  1 second audio samples created successfully


In [6]:
X,y,Z = build_dataset(INPUT_PATH )

1761  1 second audio samples created successfully


In [39]:
input_shape =  (X.shape[1],X.shape[2],1)
output =  len(CLASSES)
X = X[...,np.newaxis]

In [40]:
X = (X -  X.mean(axis=0))/ X.std(axis=0)

In [41]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.1, shuffle=True)

In [42]:
base_model = VGG16(weights=None, include_top=False, input_shape=input_shape)
x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(len(CLASSES), activation='softmax')(x)
model = keras.Model(inputs=base_model.input, outputs=x)
optim_params = dict(learning_rate = 0.003,momentum = 0.9394867962846013,decay = 0.0001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.SGD(**optim_params),metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy"),keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='sparse_top_5_categorical_accuracy')])

history =  model.fit(x=X_train,y=y_train, batch_size=2, epochs=20, validation_split=0.1,verbose=1)

model.evaluate(x=X_test,y=y_test)

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]